In [ ]:
#hi somebody deleted the assignment file completely,so couldnt access the file at all because i think editor access was given to all.Moreover we should get 72 hours for this project as stated but I got less than 19hrs,as we got this project yesterday evening plus dont have the assignment anymore so this all i can do.Also my api limit got exceeded so i requested for more tokens but it will also take time and simce i have to submit this immediately i dont have time to wait,thats why these errors at the very last is coming.Thank you for understanding!!

In [ ]:
!pip install requests beautifulsoup4 pandas pydantic python-dotenv tqdm

In [ ]:
#Please read this entire comment
#Hi somebody deleted the assignment file completely,so couldnt access the file at all because i think editor access was given to all.Moreover we should get 72 hours for this project as stated but I got less than 19hrs,as we got this project yesterday evening plus dont have the assignment anymore so this all i can do.Also my api limit got exceeded so i requested for more tokens but it will also take time and simce i have to submit this immediately i dont have time to wait,thats why these errors at the very last is coming.Thank you for understanding!!
import os, time, json, re, ast
from typing import List, Dict, Any, Tuple
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pydantic import BaseModel, Field, ValidationError
from tqdm.notebook import tqdm
from typing import Optional
os.makedirs("debug_outputs", exist_ok=True)




In [ ]:
from getpass import getpass
print("Enter your LongCat API key (starts with lc_...):")
LONGCAT_API_KEY = getpass("LONGCAT_API_KEY (input hidden): ")
if not LONGCAT_API_KEY:
    raise RuntimeError("LongCat API key not provided. Get one from LongCat dashboard.")

HEADERS = {"Authorization": f"Bearer {LONGCAT_API_KEY}", "Content-Type": "application/json"}
LONGCAT_API_URL = "https://api.longcat.chat/openai/v1/chat/completions"
MODEL_NAME = "LongCat-Flash-Chat"   # update based on your dashboard


Enter your LongCat API key (starts with lc_...):
LONGCAT_API_KEY (input hidden): ··········


In [ ]:
def fetch_url_text(url: str, timeout=20) -> str:
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers, timeout=timeout)

    if resp.status_code == 403:
        print("⚠️ 403 Forbidden, skipping:", url)
        return ""

    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    paras = soup.find_all("p")
    text = "\n".join([p.get_text().strip() for p in paras])
    text = re.sub(r'\s+', ' ', text).strip()
    return text



In [ ]:
class EntityWithAttr(BaseModel):
    entity: str = Field(..., description="Exact entity string found")
    attr_type: str = Field(..., description="Semantic type: Concept, Crop, Process, Measurement, Disease, Drug, Person, Organization, Location, Material, Other")


In [ ]:
import ast, html, math
import itertools

def _strip_markdown_and_text(raw: str) -> str:

    cleaned = re.sub(r'```(?:json|js|python)?\s*', '', raw, flags=re.IGNORECASE)
    cleaned = cleaned.replace('```', '')

    cleaned = re.sub(r'^\s*(Answer:|Response:)\s*', '', cleaned, flags=re.IGNORECASE)

    cleaned = re.sub(r'^\s*`{3}.*$', '', cleaned, flags=re.MULTILINE)

    return cleaned.strip()

def _largest_json_like_substring(s: str) -> Optional[str]:
    """
    Heuristic: find the longest substring that starts with '[' or '{' and ends with the matching bracket.
    Searches for top N candidate starts.
    """
    starts = [m.start() for m in re.finditer(r'[\[\{]', s)]
    best = None
    for i in starts[-40:]:
        start_char = s[i]
        closing = ']' if start_char == '[' else '}'
        j = s.rfind(closing)
        if j > i:
            candidate = s[i:j+1]

            if len(candidate) > 10 and (best is None or len(candidate) > len(best)):
                best = candidate
    return best or None

def tolerant_json_loads(raw: str) -> Optional[Any]:
    """
    Extremely robust JSON parser for LLM-generated outputs.
    Recovers from:
      - Markdown fences (```json)
      - Extra text before/after JSON
      - Truncated JSON endings
      - Mismatched quotes
      - Python literals (None, True, False)
      - Trailing commas
      - Single-quoted JSON-like objects
    """
    if raw is None or not isinstance(raw, str) or raw.strip() == "":
        return None

    original = raw


    cleaned = re.sub(r"```(?:json|python|js)?", "", raw, flags=re.IGNORECASE)
    cleaned = cleaned.replace("```", "")
    cleaned = re.sub(r"^\s*(Answer|Response|Output):\s*", "", cleaned, flags=re.IGNORECASE)


    first_brace = min(
        [i for i in [cleaned.find("{"), cleaned.find("[")] if i != -1] or [-1]
    )
    last_brace = max(cleaned.rfind("}"), cleaned.rfind("]"))

    if first_brace != -1 and last_brace != -1:
        slice_json = cleaned[first_brace:last_brace + 1]
    else:
        slice_json = cleaned


    slice_json = re.sub(r"\bNone\b", "null", slice_json)
    slice_json = re.sub(r"\bTrue\b", "true", slice_json)
    slice_json = re.sub(r"\bFalse\b", "false", slice_json)


    slice_json = re.sub(r"[\x00-\x1F]+", " ", slice_json)


    slice_json = re.sub(r",\s*(}|])", r"\1", slice_json)

    if slice_json.count("'") > slice_json.count('"'):
        # Replace dictionary-like expressions
        slice_json = re.sub(r"'([^']*)'\s*:", r'"\1":', slice_json)
        slice_json = re.sub(r":\s*'([^']*)'", r': "\1"', slice_json)


    try:
        return json.loads(slice_json)
    except Exception:
        pass

    try:
        return ast.literal_eval(slice_json)
    except Exception:
        pass


    if slice_json.count("{") > slice_json.count("}"):
        slice_json += "}" * (slice_json.count("{") - slice_json.count("}"))

    if slice_json.count("[") > slice_json.count("]"):
        slice_json += "]" * (slice_json.count("[") - slice_json.count("]"))

    try:
        return json.loads(slice_json)
    except Exception:
        pass

    return None


In [ ]:
def call_llm(prompt: str, max_tokens: int = 4096, temperature: float = 0.0) -> str:
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": temperature
    }

    resp = requests.post(
        LONGCAT_API_URL,
        headers=HEADERS,
        json=payload,
        timeout=120
    )

    if resp.status_code != 200:
        raise RuntimeError(f"LLM request failed: {resp.status_code} {resp.text}")

    data = resp.json()
    return data["choices"][0]["message"]["content"]


In [ ]:
ENTITY_PROMPT_TEMPLATE = """
You are an entity extraction assistant. Input paragraph:
\"\"\"{paragraph}\"\"\"

Extract up to 30 named entities from the paragraph. Return ONLY a JSON array of objects with fields:
 - "entity": exact string found in the text
 - "attr_type": one of: Concept, Crop, Process, Measurement, Disease, Drug, Person, Organization, Location, Material, Other

Example:
[{{"entity":"sustainable agriculture", "attr_type":"Concept"} , ...]]

Do NOT return any explanation or other text. If none, return [].
"""
DEDUP_PROMPT = """
You are a deduplication assistant.

Given this list of strings:
{items}

Return ONLY valid JSON with this EXACT structure:
{
  "deduplicated": [... up to 20 items max ...],
  "clusters": [... up to 20 items max ...],
  "confidence": 1.0
}

Rules:
- NEVER output more than 20 deduplicated items.
- NEVER output more than 20 clusters.
- Use ONLY double quotes.
- DO NOT include explanations or text before or after.
- If the input list is large, choose the 20 most representative distinct groups.
"""

TRIPLE_PROMPT = """
Extract relationships/triples from the paragraph using only entities from this list: {entity_list}
Return ONLY a JSON array of objects like:
[{{"src":"entity A","label":"relation short","dst":"entity B"} , ...]]
Truncate labels to 40 chars. Only include triples where both src and dst are in the provided entity list.
Return ONLY valid JSON. DO NOT include any explanation, Markdown fences, backticks, or single quotes. Use DOUBLE quotes for JSON keys and strings.

"""
TAGTYPE_PROMPT = """
Classify each entity in the input list into one of: Concept, Crop, Process, Measurement, Disease, Drug, Person, Organization, Location, Material, Other.
Input: {items}
Return ONLY a JSON array: [{{"entity":"...","tag_type":"..." }}, ...]
"""


In [ ]:
def extract_entities_from_text(paragraph: str, max_retries=3) -> List[EntityWithAttr]:

    prompt = ENTITY_PROMPT_TEMPLATE.replace("{paragraph}", paragraph[:4000])
    for attempt in range(max_retries):
        raw = call_llm(prompt)
        try:
            arr = json.loads(raw)
            validated = []
            for item in arr:
                validated.append(EntityWithAttr(**item))
            return validated
        except Exception as e:

            try:
                jstart = raw.find('[')
                jend = raw.rfind(']') + 1
                if jstart != -1 and jend != -1:
                    arr = json.loads(raw[jstart:jend])
                    validated = [EntityWithAttr(**item) for item in arr]
                    return validated
            except Exception:
                pass
            print(f"Attempt {attempt+1} parse failed: {e}; retrying...")
            time.sleep(1+attempt*2)
    return []



In [ ]:
def deduplicate_with_lm(items: List[str],
                        target_confidence: float = 0.9,
                        max_iters: int = 4
                       ) -> Tuple[List[str], List[Dict[str, Any]]]:


    items = [it for it in dict.fromkeys(items) if it.strip()]
    if not items:
        return [], []


    if len(items) > 50:
        items = items[:50]   # keep only first 50 for dedup

    for it_num in range(max_iters):


        prompt = (
            "You are a deduplication assistant.\n"
            f"Here is the list of items:\n{json.dumps(items)}\n\n"
            "Return ONLY valid JSON in this exact structure:\n"
            "{\n"
            '  "deduplicated": [... up to 20 items ...],\n'
            '  "clusters": [... up to 20 items ...],\n'
            '  "confidence": 1.0\n'
            "}\n\n"
            "Rules:\n"
            "- Never output more than 20 deduplicated items.\n"
            "- Never output more than 20 clusters.\n"
            "- Use ONLY double quotes.\n"
            "- Output ONLY JSON. No markdown, no comments.\n"
        )

        raw = call_llm(prompt, max_tokens=4096)   # increased token limit


        if raw.count("{") != raw.count("}"):
            raw = raw[:raw.rfind("}") + 1]


        parsed = tolerant_json_loads(raw)

        if parsed is None:

            idx = int(time.time() * 1000) % 1000000
            fname = f"debug_outputs/dedup_raw_{idx}.txt"
            with open(fname, "w", encoding="utf-8") as fh:
                fh.write("==== RAW OUTPUT ====\n" + raw + "\n\n")
                fh.write("==== RAW LENGTH ====\n" + str(len(raw)) + "\n\n")
            print(f"Dedup parse failed: saved raw output to {fname}")
            time.sleep(1 + it_num)
            continue


        if isinstance(parsed, dict):
            dedup = parsed.get("deduplicated", [])[:20]   # enforce limit
            clusters = parsed.get("clusters", [])[:20]
            confidence = float(parsed.get("confidence", 0) or 0)

            if confidence >= target_confidence:
                return dedup, clusters
            else:
                print(f"Dedup confidence {confidence} < {target_confidence}, iter {it_num+1}")
                time.sleep(1 + it_num)
                continue


        if isinstance(parsed, list):
            parsed = parsed[:20]
            clusters = [{"canonical": x, "members": [x]} for x in parsed]
            return parsed, clusters


        print("Dedup returned unexpected type:", type(parsed))
        time.sleep(1 + it_num)


    norm = {}
    for it in items:
        key = re.sub(r"\W+", "", it).lower()
        norm.setdefault(key, []).append(it)

    dedup = [members[0] for members in norm.values()]
    clusters = [{"canonical": grp[0], "members": grp} for grp in norm.values()]
    return dedup[:20], clusters[:20]


In [ ]:
def extract_triples(paragraph: str, entity_list: List[str]) -> List[Dict[str,str]]:
    if not entity_list:
        return []
    prompt = TRIPLE_PROMPT.replace("{entity_list}", json.dumps(entity_list))
    prompt = prompt + "\n\nParagraph:\n" + paragraph[:4000]
    raw = call_llm(prompt)
    parsed = tolerant_json_loads(raw)
    if parsed is None:
        idx = int(time.time()*1000) % 1000000
        fname = f"debug_outputs/triples_raw_{idx}.txt"
        with open(fname, "w", encoding="utf-8") as fh:
            fh.write("==== RAW OUTPUT ====\n")
            fh.write(raw[:20000])
        print(f"Triple parse failed: saved raw output to {fname}")
        return []
    triples = []
    try:
        entity_set = {e.strip().lower() for e in entity_list}
        for t in parsed:
            if not isinstance(t, dict):
                continue
            src = (t.get("src","") or "").strip()
            dst = (t.get("dst","") or "").strip()
            lbl = (t.get("label","") or "").strip()[:40]
            if src and dst and src.lower() in entity_set and dst.lower() in entity_set:
                triples.append({"src": src, "label": lbl, "dst": dst})
    except Exception as e:
        print("Triple post-processing failed:", e)
        return []
    return triples


In [ ]:
def _clean(s: str) -> str:
    return s.replace('"','\'').strip()

def triples_to_mermaid(triples: List[Dict[str,str]], entity_list: List[str]) -> str:
    entity_set = {e.strip().lower() for e in entity_list}
    lines = ["graph LR"]
    for t in triples:
        src = t["src"].strip(); dst = t["dst"].strip()
        if src.lower() in entity_set and dst.lower() in entity_set:
            lbl = t["label"][:40].replace("--","-")
            lines.append(f'    "{_clean(src)}" --|"{_clean(lbl)}"| "{_clean(dst)}"')
    return "\n".join(lines)


In [ ]:
def process_url(url: str, paragraph_limit: int = 30) -> Dict[str,Any]:
    print("Processing:", url)
    text = fetch_url_text(url)

    paragraphs = [p.strip() for p in re.split(r'\.\s+', text) if len(p.strip())>100][:paragraph_limit]
    all_entities = []
    entity_objs = []
    for p in tqdm(paragraphs, desc="Extracting entities"):
        ents = extract_entities_from_text(p)
        for e in ents:
            all_entities.append(e.entity.strip())
            entity_objs.append({"entity": e.entity.strip(), "attr_type": e.attr_type})
    deduped, clusters = deduplicate_with_lm(all_entities)
    triples = extract_triples(text, deduped)
    mermaid = triples_to_mermaid(triples, deduped)
    return {"url": url, "deduped": deduped, "clusters": clusters, "triples": triples, "mermaid": mermaid, "entity_objs": entity_objs}


In [ ]:
URLS = [
 "https://en.wikipedia.org/wiki/Sustainable_agriculture",
 "https://www.nature.com/articles/d41586-025-03353-5",
 "https://www.sciencedirect.com/science/article/pii/S1043661820315152",
 "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/",
 "https://www.fao.org/3/y4671e/y4671e06.htm",
 "https://www.medscape.com/viewarticle/time-reconsider-tramadol-chronic-pain-2025a1000ria",
 "https://www.sciencedirect.com/science/article/pii/S0378378220307088",
 "https://www.frontiersin.org/news/2025/09/01/rectangle-telescope-finding-habitable-planets",
 "https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7",
 "https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india"
]

results = []
for i, url in enumerate(URLS, start=1):
    try:
        res = process_url(url)
        results.append(res)

        fname = f"mermaid_{i}.md"
        with open(fname, "w", encoding="utf-8") as f:
            f.write(res["mermaid"])
        print("Saved:", fname)
    except Exception as e:
        print("Error processing", url, ":", e)


rows = []
for r in results:
    for ent in r["deduped"]:
        rows.append({"link": r["url"], "tag": ent, "tag_type": "Unknown"})
df = pd.DataFrame(rows)
df.to_csv("tags.csv", index=False)
print("Saved tags.csv with", len(df), "rows")


Processing: https://en.wikipedia.org/wiki/Sustainable_agriculture


Extracting entities:   0%|          | 0/30 [00:00<?, ?it/s]

Error processing https://en.wikipedia.org/wiki/Sustainable_agriculture : LLM request failed: 429 {"error":{"message":"AppId:**vu9n 达到使用量上限","type":"rate_limit_error","code":"too_many_requests"}}
Processing: https://www.nature.com/articles/d41586-025-03353-5


Extracting entities:   0%|          | 0/12 [00:00<?, ?it/s]

Error processing https://www.nature.com/articles/d41586-025-03353-5 : LLM request failed: 429 {"error":{"message":"AppId:**vu9n 达到使用量上限","type":"rate_limit_error","code":"too_many_requests"}}
Processing: https://www.sciencedirect.com/science/article/pii/S1043661820315152
⚠️ 403 Forbidden, skipping: https://www.sciencedirect.com/science/article/pii/S1043661820315152


...

Error processing https://www.frontiersin.org/news/2025/09/01/rectangle-telescope-finding-habitable-planets : LLM request failed: 429 {"error":{"message":"AppId:**vu9n 达到使用量上限","type":"rate_limit_error","code":"too_many_requests"}}
Processing: https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7


Extracting entities:   0%|          | 0/23 [00:00<?, ?it/s]

Error processing https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7 : LLM request failed: 429 {"error":{"message":"AppId:**vu9n 达到使用量上限","type":"rate_limit_error","code":"too_many_requests"}}
Processing: https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india


Extracting entities:   0%|          | 0/30 [00:00<?, ?it/s]

Error processing https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india : LLM request failed: 429 {"error":{"message":"AppId:**vu9n 达到使用量上限","type":"rate_limit_error","code":"too_many_requests"}}
Saved tags.csv with 0 rows
